In [ ]:
# imports and settings

import os
import time
import pickle
import warnings
import pandas as pd
import networkx as nx
from networkx.algorithms.coloring import greedy_color
import matplotlib.pyplot as plt
from copy import deepcopy

import numpy as np
from numpy import linalg as LA
from numpy import histogram2d

from scipy import signal
from scipy.fft import fft, fftfreq, fftshift
from scipy.signal import find_peaks, butter, filtfilt, welch, get_window
from scipy.ndimage import gaussian_filter
from scipy.io import wavfile
from scipy.stats import wasserstein_distance_nd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics.pairwise import rbf_kernel, polynomial_kernel, linear_kernel

import utils as ut
%load_ext autoreload
%autoreload 2

# do not show warnings
warnings.filterwarnings("ignore")

print("Imports complete.")

$$
\frac{d^2}{dt^2} e^{ip(t)} = (ip''(t)-p'(t))^2e^{ip(t)} 
$$

In [ ]:
def second_derivative_complex_exp(P, dt):
    """
    Calculates the second derivative of exp(i * P) w.r.t. t.
    
    Parameters:
    P  : numpy array of values p(t)
    dt : scalar, the time step between samples
    """
    # 1. Calculate the complex exponential array
    f = np.exp(1j * P)
    
    # 2. Calculate p'(t) and p''(t) using central differences
    p_prime = np.gradient(P, dt)
    p_double_prime = np.gradient(p_prime, dt)
    
    # 3. Apply the derived formula: [i*p''(t) - (p'(t))^2] * exp(i*p(t))
    second_deriv = (1j * p_double_prime - p_prime**2) * f
    
    return second_deriv

# --- Example Usage ---
t = np.linspace(0, 10, 1000)
dt = t[1] - t[0]
P = np.sin(t)  # Example function p(t) = sin(t)

d2f_dt2 = second_derivative_complex_exp(P, dt)

In [ ]:
# settings
fs = 8000  # sampling frequency
f0 = 2000  # signal frequency
duration = 60  # seconds
snr_range = (10, -10)  # from 10 dB to -10 dB
snr_vec = np.arange(snr_range[0], snr_range[1] - 1, -1)
num_thresholds = 10
num_iterations = 10

# welch args
nperseg = 1024
noverlap = 0.5
window = 'hanning'
dc = 20
crop_freq = None
norm_size = 5

# s2g args
quantization_bins = 10

# raw signal generation and spectrogram calculation
raw_signal = ut.simulate_raw_signal(f0, fs, duration)
F, T, raw_Sxx, raw_phasogram = ut.calc_spectrogram(raw_signal, fs, nperseg=nperseg, percent_overlap=noverlap, window=window, remove_dc=dc, crop_freq=crop_freq)

SNR = snr_vec
welch_threshold = 0.04
s2g_threshold_wasserstein = 0.8
s2g_threshold_edge_count = 0.1
s2g_threshold_laplacian = 0.8

welch_detector = ut.WelchDetector(fs, nperseg, noverlap, window, dc, crop_freq, norm_size)
s2g_detector_wasserstein = ut.S2GDetector(fs, nperseg, noverlap, window, dc, crop_freq, quantization_bins, mode="wasserstein")
s2g_detector_edge_count = ut.S2GDetector(fs, nperseg, noverlap, window, dc, crop_freq, quantization_bins, mode="edge_count")
s2g_detector_laplacian = ut.S2GDetector(fs, nperseg, noverlap, window, dc, crop_freq, quantization_bins, mode="laplacian")


_noise_greneration_time = []
_spectrogram_time = []
_welch_time = []
_welch_detection_time = []
_s2g_detection_time_wass = []
_s2g_detection_time_edge = []
_s2g_detection_time_lap = []

fig = make_subplots(rows=6, cols=len(SNR), shared_xaxes=False, column_titles=[f"{snr} dB" for snr in SNR])
for i, snr in enumerate(SNR, start=1):
    _t = time.time()
    rx = ut.add_noise_to_signal(raw_signal, snr_db=snr, fs=fs, signal_bw=1, noise_type='white')
    _noise_greneration_time.append(time.time() - _t)
    fig.add_trace(go.Heatmap(z=raw_Sxx, x=T, y=F, colorscale='Viridis', showlegend=False, showscale=False), row=1, col=i)

    _t = time.time()
    _F, _T, rx_Sxx, _phasogram = ut.calc_spectrogram(rx, fs, nperseg=nperseg, percent_overlap=noverlap, window=window, remove_dc=dc, crop_freq=crop_freq)
    _spectrogram_time.append(time.time() - _t)
    fig.add_trace(go.Heatmap(z=rx_Sxx, x=_T, y=_F, colorscale='Viridis', showlegend=False, showscale=False), row=2, col=i)

    _t = time.time()
    rx_pxx = ut.calc_welch_from_spectrogram(rx_Sxx, normalization_window_size=norm_size)
    _welch_time.append(time.time() - _t)
    fig.add_trace(go.Scatter(y=rx_pxx, x=_F, showlegend=False, line=dict(color='blue')), row=3, col=i)
    
    _t = time.time()
    _, welch_detections = welch_detector.detect(rx, threshold=welch_threshold)
    _welch_detection_time.append(time.time() - _t)
    if len(welch_detections) > 0:
        fig.add_trace(go.Scatter(y=rx_pxx[welch_detections], x=_F[welch_detections], mode='markers', marker=dict(color='red', size=10), showlegend=False), row=3, col=i)

    _t = time.time()
    _, s2g_detections, K = s2g_detector_wasserstein.detect(rx, threshold=s2g_threshold_wasserstein)
    _s2g_detection_time_wass.append(time.time() - _t)
    fig.add_trace(go.Scatter(y=K, x=F, showlegend=False, line=dict(color='purple')), row=4, col=i)
    if len(s2g_detections) > 0:
        fig.add_trace(go.Scatter(y=K[s2g_detections], x=F[s2g_detections], mode='markers', marker=dict(color='red', size=10), showlegend=False), row=4, col=i)

    _t = time.time()
    _, s2g_detections, K = s2g_detector_edge_count.detect(rx, threshold=s2g_threshold_edge_count)
    _s2g_detection_time_edge.append(time.time() - _t)
    fig.add_trace(go.Scatter(y=K, x=F, showlegend=False, line=dict(color='purple')), row=5, col=i)
    if len(s2g_detections) > 0:
        fig.add_trace(go.Scatter(y=K[s2g_detections], x=F[s2g_detections], mode='markers', marker=dict(color='red', size=10), showlegend=False), row=5, col=i)

    _t = time.time()
    _, s2g_detections, K = s2g_detector_laplacian.detect(rx, threshold=s2g_threshold_laplacian)
    _s2g_detection_time_lap.append(time.time() - _t)
    fig.add_trace(go.Scatter(y=K, x=F, showlegend=False, line=dict(color='purple')), row=6, col=i)
    if len(s2g_detections) > 0:
        fig.add_trace(go.Scatter(y=K[s2g_detections], x=F[s2g_detections], mode='markers', marker=dict(color='red', size=10), showlegend=False), row=6, col=i)


print("Noise generation times:", np.average(_noise_greneration_time))
print("Spectrogram calculation times:", np.average(_spectrogram_time))
print("Welch calculation times:", np.average(_welch_time))
print("Welch detection times:", np.average(_welch_detection_time))
print("S2G detection times (Wasserstein):", np.average(_s2g_detection_time_wass))
print("S2G detection times (Edge Count):", np.average(_s2g_detection_time_edge))
print("S2G detection times (Laplacian):", np.average(_s2g_detection_time_lap))

fig.update_layout(height=800, width=300*len(SNR), title_text=f"Raw vs Noisy Signal")
fig.show()

In [ ]:
SNR = -5
rx = ut.add_noise_to_signal(raw_signal, snr_db=SNR, fs=fs, signal_bw=1, noise_type='white')

_transition_matrix_time = []
_K_calculation_time = []
_L_calculation_time = []
_spectrogram_time = []

_t = time.time()
F, T, rx_Sxx, rx_phasogram = ut.calc_spectrogram(rx, fs, nperseg=nperseg, percent_overlap=noverlap/nperseg, window=window, remove_dc=dc, crop_freq=crop_freq)
_spectrogram_time.append(time.time() - _t)

uniform_M = np.ones((quantization_bins, quantization_bins)) / (quantization_bins**2)

for f_ix, f in enumerate(F):
    phase = rx_phasogram[f_ix, :]
    _t = time.time()
    transition_matrix = ut.get_s2g(phase, quantization_bins)
    _transition_matrix_time.append(time.time() - _t)
    _t = time.time()
    K = ut.get_K(transition_matrix, mode="wasserstein", uniform_M=uniform_M)
    _K_calculation_time.append(time.time() - _t)
    _t = time.time()
    L = ut.get_K(transition_matrix, mode="laplacian")
    _L_calculation_time.append(time.time() - _t)

print("Spectrogram calculation time:", np.sum(_spectrogram_time))
print("S2G calculation time:", np.sum(_transition_matrix_time))
print("K calculation time:", np.sum(_K_calculation_time))
print("L calculation time:", np.sum(_L_calculation_time))

f0_ix = np.argmin(np.abs(F - f0))
phase = rx_phasogram[f0_ix, :]
transition_matrix = ut.get_s2g(phase, quantization_bins)
K = ut.get_K(transition_matrix, mode="wasserstein")
L = ut.get_K(transition_matrix, mode="laplacian")

fig = make_subplots(rows=1, cols=2, subplot_titles=["S2G Transition Matrix", "S2G Graph"])
fig.add_trace(go.Heatmap(z=transition_matrix, colorscale='Viridis', showlegend=False, showscale=False), row=1, col=1)
G = ut.get_s2g_graph(phase, quantization_bins)
fig_g = ut.draw_graph(G)
for tr in fig_g.data:
            fig.add_trace(deepcopy(tr), row=1, col=2)
fig.update_layout(height=400, width=800, title_text=f"S2G for Signal (SNR={SNR} dB) with (K={K:.4f}) and (L={L:.4f})")
fig.show()

In [ ]:
SNR = -5
n_bins = 100
f0_ix = np.argmin(np.abs(F - f0))
rx = ut.add_noise_to_signal(raw_signal, snr_db=SNR, fs=fs, signal_bw=1, noise_type='white')

# The row indices you used from rx_phasogram
row_indices = [f0_ix, 10, 50, 100, 150, 200, 300]

# Base titles for the 4 columns
col_base_titles = [
    "Phase Dist.", 
    "Phase 1st Deriv. Dist.", 
    "Phase 2nd Deriv. Dist.", 
    "Phase 2nd Deriv. Analytical"
]
colors = ['green', 'blue', 'orange', 'red']

plot_data = []
subplot_titles = []

# 1. Calculate all data and generate dynamic titles with Mean and Std
for idx in row_indices:
    # Get raw data
    phase = rx_phasogram[idx, :]
    phase_diff1 = np.gradient(phase)
    phase_diff2 = np.gradient(phase_diff1)
    phase_diff2_ana = second_derivative_complex_exp(phase, T[1] - T[0]).real
    
    row_data = [phase, phase_diff1, phase_diff2, phase_diff2_ana]
    plot_data.append(row_data)
    
    # Calculate stats for the title captions
    for col_idx, data in enumerate(row_data):
        mean_val = np.mean(data)
        std_val = np.std(data)
        
        # Create a title with the base name and the statistics (using <br> for a new line)
        title = f"{col_base_titles[col_idx]}<br><span style='font-size:12px'>Mean: {mean_val:.2f} | Std: {std_val:.2f}</span>"
        subplot_titles.append(title)

# 2. Create the subplot figure with the 16 dynamic titles
fig = make_subplots(
    rows=len(row_indices), cols=4, 
    subplot_titles=subplot_titles,
    vertical_spacing=0.05 # Added a bit of spacing to accommodate the 2-line titles
)

# 3. Compute histograms and plot the bars
for row_idx, row_data in enumerate(plot_data):
    for col_idx, data in enumerate(row_data):
        
        # Apply the specific range (0, 2*np.pi) ONLY for the first column (Phase) as in your original code
        # if col_idx == 0:
        #     hist_counts, _ = np.histogram(data, bins=n_bins, range=(0, 2*np.pi))
        # else:
        #     hist_counts, _ = np.histogram(data, bins=n_bins)
        
        hist_counts, hist_bins = np.histogram(data, bins=n_bins)
            
        fig.add_trace(
            go.Bar(
                x=hist_bins[:-1], 
                y=hist_counts, 
                marker_color=colors[col_idx], 
                showlegend=False
            ), 
            row=row_idx + 1, col=col_idx + 1
        )

# 4. Update final layout
fig.update_layout(
    width=1200, 
    height=200*len(row_indices), # Increased height slightly for better readability with the new titles
    title=f"Phase and Derivative Distributions for f={f0} Hz (SNR={SNR} dB)"
)

fig.show()

In [ ]:
SNR = -5
rx = ut.add_noise_to_signal(raw_signal, snr_db=SNR, fs=fs, signal_bw=1, noise_type='white')
F, T, rx_Sxx, rx_phasogram = ut.calc_spectrogram(rx, fs, nperseg=nperseg, percent_overlap=noverlap/nperseg, window=window, remove_dc=dc, crop_freq=crop_freq)
pxx = ut.calc_welch_from_spectrogram(rx_Sxx, normalization_window_size=norm_size)
phasogram_std = np.std(rx_phasogram, axis=1)
phasogram_gradient = np.gradient(rx_phasogram, axis=1)
phasogram_gradient_std = np.std(phasogram_gradient, axis=1)
K = ut.get_all_Ks(rx_phasogram, F, quantization_bins, mode="laplacian")

fig = make_subplots(rows=1, cols=5, subplot_titles=["Spectrogram", "Welch PSD", "Phasogram Std Dev", "Phasogram Gradient Std Dev", "K Values"], shared_yaxes=True, column_widths=[0.4, 0.2, 0.2, 0.2, 0.2])
fig.add_trace(go.Heatmap(z=rx_Sxx, x=T, y=F, colorscale='Viridis', showlegend=False, showscale=False), row=1, col=1)
fig.add_trace(go.Scatter(y=F, x=pxx, showlegend=False, line=dict(color='blue')), row=1, col=2)
fig.add_trace(go.Scatter(y=F, x=phasogram_std, showlegend=False, line=dict(color='orange')), row=1, col=3)
fig.add_trace(go.Scatter(y=F, x=phasogram_gradient_std, showlegend=False, line=dict(color='red')), row=1, col=4)
fig.add_trace(go.Scatter(y=F, x=K, showlegend=False, line=dict(color='green')), row=1, col=5)
fig.update_layout(height=400, width=1200, title_text=f"Spectrogram and Derived Metrics for Noisy Signal (SNR={SNR} dB)")
fig.show()